In [18]:
pip install dask[complete]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#5.1 Параллельные DataFrame и pandas

In [19]:
import dask.dataframe as dd
from dask.distributed import Client


In [20]:
client = Client()
client


C:\Users\Ilvin\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52763 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52763/status,
Dashboard: http://127.0.0.1:52763/status,Workers: 4
Total threads: 4,Total memory: 3.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52764,Workers: 4
Dashboard: http://127.0.0.1:52763/status,Total threads: 4
Started: Just now,Total memory: 3.88 GiB
Comm: tcp://127.0.0.1:52788,Total threads: 1
Dashboard: http://127.0.0.1:52790/status,Memory: 0.97 GiB
Nanny: tcp://127.0.0.1:52767,


In [21]:
ddf = dd.read_parquet(
    "s3://dask-data/nyc-taxi/nyc-2015.parquet/part.*.parquet",
    storage_options={"anon": True})

In [24]:
ddf.columns

Index([], dtype='object')

In [ ]:
ddf.groupby(
'passenger_count').agg(
{'tip_amount':'mean'}).head(10)

In [27]:
#5.2 
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

C:\Users\Ilvin\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52978 instead
  warnings.warn(
C:\Users\Ilvin\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.03s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.20.10.3:52978/status,
Dashboard: http://172.20.10.3:52978/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://172.20.10.3/18008/1,Workers: 1
Dashboard: http://172.20.10.3:52978/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://172.20.10.3/18008/4,Total threads: 4
Dashboard: http://172.20.10.3:52979/status,Memory: 1.86 GiB
Nanny: None,


In [28]:
import dask.array as da
w1 = da.random.random((256, 10000), chunks=(256, 1000))*0.00001
w1


dask.array<mul, shape=(256, 10000), dtype=float64, chunksize=(256, 1000), chunktype=numpy.ndarray>

In [29]:
b1 = da.random.random((10000,),chunks=(1000,))*0.00001
b1

dask.array<mul, shape=(10000,), dtype=float64, chunksize=(1000,), chunktype=numpy.ndarray>

In [30]:
w2 = da.random.random((10000, 256), chunks=(1000, 256))*0.00001
b2 = da.random.random((256,), chunks=(256,))*0.00001

In [31]:
sigma = lambda x: 1 / (1 + da.exp(-x))

In [32]:
x = da.array([
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])
print(x.shape)

(16, 16)


In [33]:
net = lambda x: sigma(da.dot(sigma(da.dot(x, w1) + b1), w2) + b2)


In [34]:
out = net(da.reshape(x, (x.shape[0]*x.shape[1],))).compute()
out

array([0.50628961, 0.50627411, 0.50630493, 0.50628322, 0.50623289,
       0.50623041, 0.50624682, 0.50629884, 0.50626885, 0.50624648,
       0.5062143 , 0.50632804, 0.50622049, 0.5062875 , 0.50620876,
       0.50626699, 0.50625522, 0.50627322, 0.50629356, 0.50618142,
       0.50624534, 0.50622462, 0.50627586, 0.50620808, 0.50626749,
       0.50615814, 0.50622676, 0.50623327, 0.50624338, 0.50633602,
       0.50627343, 0.50628152, 0.50629265, 0.5062499 , 0.50621561,
       0.5062682 , 0.50628359, 0.50625463, 0.50626472, 0.50617649,
       0.50618943, 0.50624021, 0.50621463, 0.50627478, 0.50621795,
       0.50624725, 0.50627026, 0.5063152 , 0.50623329, 0.5062341 ,
       0.50627358, 0.50624874, 0.50627848, 0.50623852, 0.50621659,
       0.5062804 , 0.50624651, 0.50621424, 0.50620217, 0.50625185,
       0.50626662, 0.50627401, 0.50624798, 0.50627457, 0.50624095,
       0.50627041, 0.50620802, 0.50626836, 0.50622724, 0.50629083,
       0.50628349, 0.50619975, 0.50626267, 0.50616681, 0.50630